In [2]:
import tensorflow as tf
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
new_model = tf.keras.models.load_model('sentimentmodel.h5')
new_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 50, 64)         │     1,280,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 48, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_6          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,305,029 (4.98 MB)

 Trainable params: 1,305,026 (4.98 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3 (16.00 B)

In [4]:
with open('tokenizer.json') as json_file:
    json_string = json_file.read()

tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string)

In [5]:
# text = "I am doing alright"

In [6]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()


text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def textcleaning(text, stem=False):
    tokens = []
    text = re.sub(text_cleaning_re, ' ', str(text)).strip()
    
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            if lemmatizer:
                tokens.append(lemmatizer.lemmatize(token))
            else:
                tokens.append(token)
                
                
    return " ".join(tokens)
                


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [72]:
text = "I've been feeling really overwhelmed lately because of the increased workload and constant deadlines, but when I finally managed to take a break and spend some quality time with my friends, it felt incredibly refreshing and made me realize how important it is to balance work and personal life, even though it's not always easy to do"
words_to_check = ["I", "you", "he", "she", "they", "we", "The", 'the', "I've"]  

words_in_text = text.split()


filtered_words = [word for word in words_in_text if word not in words_to_check]


new_text = " ".join(filtered_words)

print(new_text)


been feeling really overwhelmed lately because of increased workload and constant deadlines, but when finally managed to take a break and spend some quality time with my friends, it felt incredibly refreshing and made me realize how important it is to balance work and personal life, even though it's not always easy to do


In [73]:
text = new_text
list(text)
text = textcleaning(text)
print(text)
tokenizer1.fit_on_texts([text])
sequences = tokenizer1.texts_to_sequences([text])

feeling really overwhelmed lately increased workload constant deadline finally managed take break spend quality time friend felt incredibly refreshing made realize important balance work personal life even though always easy


In [74]:
sequences = pad_sequences(sequences, padding='post', maxlen=50)

In [75]:
predictions = new_model.predict(sequences)

print(predictions)
predictions.tolist

if (predictions[0][0] > 0.45 and predictions[0][0] < 0.55):
    print("calm")
else:
    index = np.argmax(predictions)
    if (index == 1):
        print("sad")
    else:
        print("happy")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.558799   0.44120094]]
good
